In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
antipsychotic_data = (pd.read_excel('data/ASD_Antipsychotics_MPH_MasterData_1.xlsx', sheetname='Antipsychotics_data', na_values=['ND','NR','NRNR'])
        .rename(columns={'FUP SAMPLE SIZE (this is how many people were assessed for that FUP at that point in that group)':'FUP_SAMPLE_SIZE',
                        "FUP_count (this is the N associated with %; eg., anxiety=10 out of 100 subjects (10%), it will be 'ND' if FUPs are reported as means , median etc)":"'FUP_COUNT"}))

In [34]:
antipsychotic_data.head()

,REFID,Citation,Child Paper,Rx Grouping,Group_Desc,Intervention_category,Dose,Route,Rx_Durn,OUTC_Main_ CATG,...,Change score_Q1,Change score_Q3,Change score Min,Change score Max,Change score 95% L,Change score 95% H,Other stats (this will be the name & values of other stats not listed in the earlier columns ),"Results (between group difference, p value, ES, OR, RR, etc)",Comments,Comments_2
0,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=1.2",NaN,NaN
1,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G2:,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=1.0",NaN,NaN
3,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G2:,Placebo,Placebo,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4412_4405_4400_4393_4396_4395_4391_4380_1483_4...,McCracken 2002,NaN,G1:,Risperidone,Medical,up to 2.5 mg/day,oral,8 wks,Symptoms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G1 vs G2, p<0.001, ES=0.8",NaN,NaN


In [35]:
antipsychotic_data.columns

Index(['REFID', 'Citation', 'Child Paper', 'Rx Grouping', 'Group_Desc',
       'Intervention_category', 'Dose', 'Route', 'Rx_Durn', 'OUTC_Main_\nCATG',
       'Outcome_Sub_category', 'Outc_Tool',
       'Outcome_specify (use same full name across rows, no abbreviations)',
       'REPORTER (who answers assessment)', 'Baseline sample size', 'BL_count',
       'BL %', 'BL Mean', 'BL SD', 'BL SE', 'BL_Median', 'BL_Q1', 'BL_Q3',
       'BL Min', 'BL Max', 'BL 95% L', 'BL 95% H',
       'Assessment tmpt (during Rx / EOT/ weeks etc)', 'sample_size', 'count',
       ' %', 'Mean', 'SD', 'SE', 'Median', 'Q1', 'Q3', 'Min', 'Max', '95% L',
       '95% H',
       'FUP timepoint (when was this FUP measured, e.g.,3m,6m,12 months after Rx, etc)',
       'FUP_SAMPLE_SIZE', ''FUP_COUNT', 'FUP _percent ', 'FUP\n Mean',
       'FUP \nSD', 'FUP \n SE', 'FUP \n Median', 'FUP\n _Q1', 'FUP \n_Q3',
       'FUP\n  Min', 'FUP \n Max', 'FUP \n 95% L', 'FUP \n 95% H',
       'Change score _Describe (EOT-BL or FUP-

In [36]:
antipsychotic_data.groupby('Outc_Tool').Outc_Tool.count().sort_values(ascending=False)

Outc_Tool
Aberrant Behavior Checklist                           215
clinical global impression (overall, irritability)    101
Other                                                  28
Children’s Yale-Brown Obsessive Compulsive Scale       26
Vineland Adaptive Behavior Scale                       22
Ritvo-Freeman Real Life Rating Scales                  12
Modiﬁed Real Life Rating Scale for Autism (M-RLRS      10
Pediatric  Quality of  Life  Inventory  (PedsQL)        8
Children’s Global Assessment Scale (CGAS)               4
CARS                                                    4
Multiple                                                2
Name: Outc_Tool, dtype: int64

In [37]:
(antipsychotic_data[antipsychotic_data.Outc_Tool=='Aberrant Behavior Checklist']
             .groupby('REFID')
             .agg({'Baseline sample size' : np.sum, 
                   'sample_size' : np.sum,
                   'FUP_SAMPLE_SIZE': np.sum,
                   'Change score sample_size': np.sum}))

,sample_size,Change score sample_size,FUP_SAMPLE_SIZE,Baseline sample size
REFID,,,,
204_805,NaN,395.0,NaN,568.0
4398_4387,539.0,385.0,NaN,962.0
4412_4405_4400_4393_4396_4395_4391_4380_1483_412_1605_1745,820.0,NaN,406.0,1226.0
4423_412_1745,1610.0,NaN,NaN,1860.0
4424_4425_840_2524_1111_713_869,NaN,3800.0,NaN,4718.0
83,205.0,205.0,NaN,425.0
90,500.0,NaN,NaN,590.0


In [38]:
(antipsychotic_data[antipsychotic_data.Outc_Tool=='clinical global impression (overall, irritability)']
             .groupby('REFID')
             .agg({'Baseline sample size' : np.sum, 
                   'sample_size' : np.sum,
                   'FUP_SAMPLE_SIZE': np.sum,
                   'Change score sample_size': np.sum}))

,sample_size,Change score sample_size,FUP_SAMPLE_SIZE,Baseline sample size
REFID,,,,
4388,156.0,NaN,NaN,156.0
204_805,92.0,92.0,NaN,184.0
4398_4387,NaN,NaN,NaN,55.0
4412_4405_4400_4393_4396_4395_4391_4380_1483_412_1605_1745,1139.0,NaN,81.0,1708.0
4423_412_1745,NaN,NaN,NaN,496.0
4424_4425_840_2524_1111_713_869,375.0,452.0,NaN,1110.0
83,41.0,NaN,NaN,85.0
90,224.0,NaN,NaN,236.0
